# Relax Take Home Challenge

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from collections import Counter
init_notebook_mode(connected=True)

In [2]:
# reading in files
users_df = pd.read_csv('takehome_users.csv', encoding = "ISO-8859-1", infer_datetime_format=True)
eng_df = pd.read_csv('takehome_user_engagement.csv', encoding = "ISO-8859-1",infer_datetime_format=True)

# correcting index, correcting datetimes
users_df['last_session_creation_time'] = pd.to_datetime(users_df['last_session_creation_time'],unit='s')
users_df['creation_time'] = pd.to_datetime(users_df['creation_time'])
users_df = users_df.set_index('object_id')

# validate
users_df.head()

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0


### Identifying User Adoption

The first thing to determine is if a user has adopted the platform or not. The definition of 'adopted' here is if the user has logged into the product on three separate days in at least one seven day period. This value is determined in the code below, which does the following:
1. Sets the index to be the timestamp of the visit
2. Defines a new series with the index set to all the user's unique ID's.
3. Loops through the df, extracting a list of timestamps for each time the user visited the platform
4. Iterates through each visit by the user to see if there were at least three visits by the user in the next seven days. If there were, set active to True, update the series, and break the loop.
5. With the series constructed, add it to the users dataframe.
5. Fill any untouched NaN values with 0, representing users who did not adopt the platform according to the criteria above or never logged in at all.

In [3]:
eng_df["time_stamp"] = pd.to_datetime(eng_df["time_stamp"])
eng_df.set_index(eng_df["time_stamp"],inplace=True)

adopted_users = pd.Series(index=eng_df.user_id.unique())
date_active = pd.Series(index=eng_df.user_id.unique())

for user in adopted_users.index:
    user_visits = eng_df.query('user_id == %d' % user).index
    for i in user_visits:
        max_day = i + dt.timedelta(days=7)
        active = user_visits[(user_visits <= max_day) & (user_visits >= i)].shape[0] >= 3
        if active:
            adopted_users[user] = True
            date_active[user] = i 
            break

users_df['date_adopted'] = date_active
users_df['adopted'] = adopted_users
users_df['adopted'].fillna(0, inplace=True)

users_df.head()

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,date_adopted,adopted
object_id,,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,NaN,0.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,2014-02-03 03:45:04,1.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0,NaN,0.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0,NaN,0.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0,NaN,0.0
6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,2013-12-19 03:37:06,0,0,197,11241.0,NaN,0.0
7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,2012-12-20 13:24:32,0,1,37,NaN,NaN,0.0
8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaT,1,1,74,NaN,NaN,0.0
9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaT,0,0,302,NaN,NaN,0.0


In [ ]:
# uses similar code to the above to build a list of the time between user visits, then
# find the median time between them
median_time_visits = pd.Series(index=eng_df.user_id.unique())

for user in adopted_users.index:
    user_visits = eng_df.query('user_id == %d' % user).index
    time_between_visits = []
    for n, i in enumerate(user_visits):
        try:
            time_between_visits.append((user_visits[n+1]-i) / np.timedelta64(1, 'D'))
        except:
            break
    if len(time_between_visits) != 0:
        median_time_visits[user] = np.median(time_between_visits)
    
users_df['median_time_visits'] = median_time_visits

In [4]:
# determine a user's total visits
users_df['total_visits'] = eng_df.pivot_table(index='user_id', values='visited', aggfunc=np.sum)
users_df['total_visits'].fillna(0, inplace=True)

In [5]:
# define the time between account creation and the beginning of the session that triggered adoption
adopted_users = users_df.query('adopted==1')
adopted_users.loc[:,'date_adopted'] = pd.to_datetime(adopted_users.loc[:,'date_adopted'])
adopted_users.loc[:,'creation_to_adoption'] = ((adopted_users.loc[:,'date_adopted'] - adopted_users.loc[:,'creation_time']) / np.timedelta64(1, 'D'))
users_df.loc[:,'creation_to_adoption'] = adopted_users.loc[:,'creation_to_adoption']

/Users/ChesterHitz/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/ChesterHitz/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## Analyzing Adopted and Non-Adopted Users

### Overall

Overall, only 13% of users end up adopting the platform according the criteria above. But of those 13%, what characteristics do they share in common?

In [6]:
'User adoption: ' + str(users_df.adopted.mean())

'User adoption: 0.138'

### Invitation

In [7]:
print('Invited users adoption rate:', users_df[users_df.invited_by_user_id.notnull()].adopted.mean())
print('Non-Invited users adoption rate:',users_df[users_df.invited_by_user_id.isnull()].adopted.mean())

Invited users adoption rate: 0.14695340501792115
Non-Invited users adoption rate: 0.1277091169622067


In [9]:
# buld dataframe to understand adoption based on creation source

columns = pd.MultiIndex.from_product([['All users', 'Adopted users'], ['Raw Count', 'Percentage']])

inv_summary = pd.DataFrame(index=users_df.creation_source.unique(), columns=columns)

inv_summary[('All users','Raw Count')] = users_df.creation_source.value_counts()
inv_summary[('All users','Percentage')] = round(users_df.creation_source.value_counts() / users_df.shape[0],2) * 100
inv_summary[('Adopted users','Raw Count')] = users_df.query('adopted==1').creation_source.value_counts()
inv_summary[('Adopted users','Percentage')] = round(users_df.query('adopted==1').creation_source.value_counts() / users_df.query('adopted==1').shape[0],2) * 100

inv_summary['test'] = inv_summary[('Adopted users','Percentage')] - inv_summary[('All users','Percentage')]

inv_summary

All users            Adopted users            test
                   Raw Count Percentage     Raw Count Percentage     
GUEST_INVITE            2163       18.0           369       22.0  4.0
ORG_INVITE              4254       35.0           574       35.0  0.0
SIGNUP                  2087       17.0           302       18.0  1.0
PERSONAL_PROJECTS       2111       18.0           172       10.0 -8.0
SIGNUP_GOOGLE_AUTH      1385       12.0           239       14.0  2.0

In [12]:
# quick analysis of the median time between visits and adoption rate
users_df.pivot_table(index='creation_source', columns='adopted', values='median_time_visits', aggfunc=np.mean)

adopted,0.0,1.0
creation_source,,
GUEST_INVITE,12.584135,2.933604
ORG_INVITE,12.471386,2.988676
PERSONAL_PROJECTS,14.192308,3.209302
SIGNUP,12.451977,2.718543
SIGNUP_GOOGLE_AUTH,14.318548,2.880753


### Duration

In [57]:
# determining the duration between the time the user signed up and signed off the platform
users_df['first_last_span'] = ((users_df['last_session_creation_time'] - users_df['creation_time']) / np.timedelta64(1, 'D')).astype(float)
users_df['first_last_span'] = users_df['first_last_span'].fillna(0).astype('int')

# translated to weeks
users_df['weeks_lasted'] = users_df['first_last_span'] // 7 

# determine users who only lasted < week
(users_df['weeks_lasted'].value_counts() / users_df.shape[0])[0]

0.78641666666666665

In [ ]:
# look at percentage adopted per week lasted until
(users_df.groupby('weeks_lasted').mean()['adopted'] > .5)[:20]

In [46]:
# plot that
max_weeks = np.max(users_df['weeks_lasted'])

# building plotly lists...
adopted = [dict(Counter([w for w in users_df.query('adopted == 1').weeks_lasted if w > 0])).get(x) for x in range(max_weeks)]
no_adopted = [dict(Counter([w for w in users_df.query('adopted == 0').weeks_lasted if w > 0])).get(x) for x in range(max_weeks)]

adopted_bar = go.Bar(
    x=np.arange(max_weeks),
    y= adopted,
    name='Adopted',
    textposition = 'auto',
    marker=dict(color='#2298c1')
)
no_adopted_bar = go.Bar(
    x=np.arange(max_weeks),
    y= no_adopted,
    name='Non Adopted',
    marker=dict(color='#c14b22')
)

data = [no_adopted_bar, adopted_bar]
layout = go.Layout(
    barmode='stack',
    title='Adoption since week of account creation',
    yaxis=dict(title='# of Users'),
    xaxis=dict(title='Week since account creation')
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

### Marketing Drip and Mailing List

In [13]:
# determine which and how many marketing subscriptions the user is subscribed to
def marketing_investigate(row):
    if row.opted_in_to_mailing_list == 0 and row.enabled_for_marketing_drip == 0:
        return 'Neither'
    elif row.enabled_for_marketing_drip == 1 and row.opted_in_to_mailing_list == 0:
        return 'Marketing Drip'
    elif row.enabled_for_marketing_drip == 0 and row.opted_in_to_mailing_list == 1:
        return 'Mailing list'
    else:
        return 'Both'
    
users_df['Marketing_Options'] = users_df.apply(marketing_investigate, axis=1).astype('category')
users_df.groupby('Marketing_Options').mean().adopted


Marketing_Options
Both              0.146051
Mailing list      0.141041
Marketing Drip    0.135556
Neither           0.136279
Name: adopted, dtype: float64

In [14]:
# make boxplot to compare a user's total visits with their marketing options
marketing_df = users_df[['Marketing_Options','adopted','total_visits']].query('adopted==1')

trace0 = go.Box(
    y=marketing_df[marketing_df.Marketing_Options == 'Neither'].total_visits,
    name='Neither',
    marker=dict(
        color='#c14b22'
    )
)

trace1 = go.Box(
    y=marketing_df[marketing_df.Marketing_Options == 'Marketing Drip'].total_visits,
    name='Marketing Drip',
    marker=dict(
        color='#f28143'
    )
)
trace2 = go.Box(
    y=marketing_df[marketing_df.Marketing_Options == 'Mailing list'].total_visits,
    name='Mailing list',
    marker=dict(
        color='#106f7c'
    )
)
trace3 = go.Box(
    y=marketing_df[marketing_df.Marketing_Options == 'Both'].total_visits,
    name='Both',
    marker=dict(
        color='#fde033'
    )
)
data = [trace0, trace1, trace2, trace3]
layout = go.Layout(
    yaxis=dict(
        title='visits',
        zeroline=False
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Time

Did not end up using this analysis, because nothing of interest was found.

In [16]:
users_df['creation_MM_YYYY'] = users_df['creation_time'].map(lambda x: ((x.year*100)+x.month))

months = np.sort(users_df.creation_MM_YYYY.unique())

adopted = [dict(Counter([ym for ym in users_df.query('adopted == 1').creation_MM_YYYY])).get(x) for x in months]
no_adopted = [dict(Counter([ym for ym in users_df.query('adopted == 0').creation_MM_YYYY])).get(x) for x in months]

In [17]:
adopted_bar = go.Bar(
    x=[str(x[4:])+'-'+str(x[:-2]) for x in months.astype(str)],
    y= adopted,
    name='Adopted',
    text = [str(round((x/(x+y))*100,1))+'%' for x,y in zip(adopted, no_adopted)],
    textposition = 'auto',
    marker=dict(color='#2298c1')
)
no_adopted_bar = go.Bar(
    x=[str(x[4:])+'-'+str(x[:-2]) for x in months.astype(str)],
    y= no_adopted,
    name='Non Adopted',
    marker=dict(color='#c14b22')
)

data = [no_adopted_bar, adopted_bar]
layout = go.Layout(
    barmode='stack',
    title='Adoption Rates over Time',
    yaxis=dict(title='# of Users'),
    xaxis=dict(title='Month and Year (MM-YYYY)')
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')